
# Quantify RNA spots that fall within tissue regions

This notebook will count the number of spots that fall within tissue based on manual annotations made in QuPath. This also excludes large artefacts or other regions in on the tissue that should not be counted as actual tissue.

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import os

In [3]:
import json
from shapely.geometry import shape, Point

def count_points_in_shape(gdf_geojson, points_df):
    # Turn points into a list of tuples
    points = [tuple(x) for x in points_df.to_numpy()]

    gdf_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in points])

    # Set the same CRS for both GeoDataFrames if they're not the same
    gdf_points.crs = gdf_geojson.crs

    # Perform spatial join
    gdf_joined = gpd.sjoin(gdf_points, gdf_geojson, op='within')

    ## Write measurements to dataframe
    total_area = gdf_geojson.geometry.area.sum() * 0.138 * 0.138
    number_of_spots = gdf_joined.shape[0]

    return [total_area, number_of_spots]

In [ ]:
## Create an empty dataframe to capture tissue area and spot count
df = pd.DataFrame(columns=['tissue_area', 'spot_count'])

## Iterate over each geojson file
geojson_dir = "../../annotations/molkart/heart_regions"
spot_dir = "../../../data/nf-core_molkart/mindagap"
# Iterate over each file in geojson_dir
for file in os.listdir(geojson_dir):
    print(file)
    sample_id = file.split(".")[1]

    # Load your GeoJSON data
    gdf_geojson = gpd.read_file(f'{geojson_dir}/{file}')

    # # Assuming you have a csv file with 'x' and 'y' columns for the pixel coordinates
    points_df = pd.read_table(f'{spot_dir}/{sample_id}.spots_markedDups.txt',
                        sep='\t', names=['x', 'y','z','gene'])
    # Subset pixels for x and y coordinates
    points_df = points_df[['x', 'y']]
    
    # Turn points into a list of tuples
    points = [tuple(x) for x in points_df.to_numpy()]

    gdf_points = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in points])

    # Set the same CRS for both GeoDataFrames if they're not the same
    gdf_points.crs = gdf_geojson.crs

    # Perform spatial join
    gdf_joined = gpd.sjoin(gdf_points, gdf_geojson, op='within')

    ## Write measurements to dataframe
    total_area = gdf_geojson.geometry.area.sum() * 0.138 * 0.138
    number_of_spots = gdf_joined.shape[0]

    # Store total_area and number_of_spots in a dictionary with sample_id as key
    df.loc[sample_id] = [total_area, number_of_spots]

In [7]:
# Turn rownames from df into a column named sample and remove rownames
df['sample'] = df.index
df.index.name = None

df['spots_per_um2'] = df['spot_count'] / df['tissue_area']
df
## Write dataframe to csv without rownames and with tab as separator
df.to_csv("../../output/molkart/molkart.spots_per_tissue.tsv", sep='\t', index=False) 